<a href="https://colab.research.google.com/github/cbeckler/eco_cancer_incidence_rates/blob/main/Machine%20Learning%20Model/model_20032016_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-20 19:09:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2023-01-20 19:09:09 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# start spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("balanced_random_forest").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#connect to database

import pandas as pd

conn_path = "/content/drive/MyDrive/Data/db_conn_info.csv"

conn_info = pd.read_csv(conn_path)

db_name = conn_info['db_name'][0]

db_pw = conn_info['password'][0]

db_host = conn_info['host'][0]

db_port = conn_info['port'][0]

db_user = conn_info['user'][0]

In [6]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql://{db_name}:{db_pw}@{db_host}:{db_port}/{db_user}')

In [7]:
# import model dataset

df = pd.read_sql("select * from model_dataset", engine)

# create list of new col names to replace specific year values

col_list = [name for name in df.columns]

new_col_names = []

for col in col_list:

  if '2001' in col:
    new_name = col.replace('2001', 'Y1')
    new_col_names.append(new_name)
  elif '2002' in col:
    new_name = col.replace('2002', 'Y2')
    new_col_names.append(new_name)
  elif '2003' in col:
    new_name = col.replace('2003', 'Y3')
    new_col_names.append(new_name)
  elif '2004' in col:
    new_name = col.replace('2004', 'Y4')
    new_col_names.append(new_name)
  elif '2005' in col:
    new_name = col.replace('2005', 'Y5')
    new_col_names.append(new_name)
  elif '2006' in col:
    new_name = col.replace('2006', 'Y6')
    new_col_names.append(new_name)
  elif '2007' in col:
    new_name = col.replace('2007', 'Y7')
    new_col_names.append(new_name)
  elif '2008' in col:
    new_name = col.replace('2008', 'Y8')
    new_col_names.append(new_name)
  elif '2009' in col:
    new_name = col.replace('2009', 'Y9')
    new_col_names.append(new_name)
  elif '2010' in col:
    new_name = col.replace('2010', 'Y10')
    new_col_names.append(new_name)
  elif '2011' in col:
    new_name = col.replace('2011', 'Y12')
    new_col_names.append(new_name)
  elif '2012' in col:
    new_name = col.replace('2012', 'Y13')
    new_col_names.append(new_name)
  elif '2013' in col:
    new_name = col.replace('2013', 'Y14')
    new_col_names.append(new_name)
  elif '2014' in col:
    new_name = col.replace('2014', 'Y15')
    new_col_names.append(new_name)
  else:
    new_col_names.append(col)

new_col_names

# apply new col names

df.columns = new_col_names

df.head()



,index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_Y1,...,O3_med_pred_Y5,O3_med_pred_Y6,O3_med_pred_Y7,O3_med_pred_Y8,O3_med_pred_Y9,O3_med_pred_Y10,O3_med_pred_Y12,O3_med_pred_Y13,O3_med_pred_Y14,O3_med_pred_Y15
0,0,1101,falling,12.555164,12.013325,11.958884,41.874165,39.939759,39.977164,13.365598,...,40.567937,43.333084,43.589498,40.750847,36.326936,41.478252,41.055674,39.470118,36.512861,37.457633
1,1,1103,stable,13.287901,12.684610,12.647047,43.308170,41.901690,41.888594,15.564124,...,43.226680,44.786660,46.470120,42.158683,38.435499,42.927004,42.164915,41.937580,38.473920,38.021493
2,2,1105,stable,11.426605,11.072150,11.079387,39.805057,38.986625,39.022229,12.768430,...,40.612558,43.447473,43.062623,38.970321,34.915892,39.396689,38.999311,37.908802,35.517161,35.654156
3,3,1107,stable,11.594499,11.146401,11.155309,39.979953,39.173487,39.137133,13.199240,...,41.574587,43.677365,42.648266,38.709791,35.148054,39.154417,38.971029,38.468993,36.022568,35.750203
4,4,1109,stable,11.352271,10.939546,10.932512,40.690546,39.856677,39.866958,12.164619,...,40.589950,43.572161,43.102934,40.463900,36.734288,41.022877,40.996091,38.420984,36.382191,37.127240


In [8]:
import numpy as np
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE


## Create Features and target variable

In [9]:
X = df.drop(['index', 'FIPS', 'recent_trend'], axis=1)
y = df['recent_trend']

In [10]:
y.value_counts()

stable     961
falling     68
rising      19
Name: recent_trend, dtype: int64

## Data split into Training and Testing

In [11]:
#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)
scaler =StandardScaler().fit(X_train)

X_train_scaled =scaler.transform(X_train)
X_test_scaled =scaler.transform(X_test)

## Data resampled with SMOTE

In [12]:
# Resampling training data with SMOTE
X_resampled, y_resampled = SMOTE(
    random_state=1,sampling_strategy='auto').fit_resample(
    X_train_scaled, y_train)

print(f'y_resampled: {Counter(y_resampled)}')

y_resampled: Counter({'stable': 721, 'falling': 721, 'rising': 721})


## Balanced Random Forest Classifier 

In [13]:
from imblearn.ensemble import BalancedRandomForestClassifier
brc=BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_resampled, y_resampled)

In [14]:
y_pred= brc.predict(X_test_scaled)

In [15]:
y_pred_train= brc.predict(X_train_scaled)

## Balanced Accuracy Score

In [16]:
# Balanced accuracy training score 
balanced_accuracy_score(y_train, y_pred_train)

1.0

In [17]:
# Balanced accuracy testign score
balanced_accuracy_score(y_test, y_pred)


0.3491013071895425

## Confusion Matrix

In [18]:
cm=confusion_matrix(y_test, y_pred)
cm

array([[  3,   1,  13],
       [  0,   0,   5],
       [ 24,   7, 209]])

In [19]:
brc_df=pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
brc_df.head()

,Prediction,Actual
0,stable,stable
1,stable,stable
2,stable,stable
3,stable,stable
4,stable,stable


In [20]:
print(brc_df.Prediction.value_counts())
print(brc_df.Actual.value_counts())

stable     227
falling     27
rising       8
Name: Prediction, dtype: int64
stable     240
falling     17
rising       5
Name: Actual, dtype: int64


## Imbalanced Classification Report

In [21]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    falling       0.11      0.18      0.90      0.14      0.40      0.15        17
     rising       0.00      0.00      0.97      0.00      0.00      0.00         5
     stable       0.92      0.87      0.18      0.90      0.40      0.17       240

avg / total       0.85      0.81      0.24      0.83      0.39      0.16       262



# Predictions with trained model on 2003-2016 data

In [22]:
# import model dataset

df_predictions = pd.read_sql("select * from prediction_model_dataset", engine)

# create list of new col names to replace specific year values

col_list = [name for name in df_predictions.columns]

new_col_names = []

for col in col_list:

  if '2003' in col:
    new_name = col.replace('2003', 'Y1')
    new_col_names.append(new_name)
  elif '2004' in col:
    new_name = col.replace('2004', 'Y2')
    new_col_names.append(new_name)
  elif '2005' in col:
    new_name = col.replace('2005', 'Y3')
    new_col_names.append(new_name)
  elif '2006' in col:
    new_name = col.replace('2006', 'Y4')
    new_col_names.append(new_name)
  elif '2007' in col:
    new_name = col.replace('2007', 'Y5')
    new_col_names.append(new_name)
  elif '2008' in col:
    new_name = col.replace('2008', 'Y6')
    new_col_names.append(new_name)
  elif '2009' in col:
    new_name = col.replace('2009', 'Y7')
    new_col_names.append(new_name)
  elif '2010' in col:
    new_name = col.replace('2010', 'Y8')
    new_col_names.append(new_name)
  elif '2011' in col:
    new_name = col.replace('2011', 'Y9')
    new_col_names.append(new_name)
  elif '2012' in col:
    new_name = col.replace('2012', 'Y10')
    new_col_names.append(new_name)
  elif '2013' in col:
    new_name = col.replace('2013', 'Y12')
    new_col_names.append(new_name)
  elif '2014' in col:
    new_name = col.replace('2014', 'Y13')
    new_col_names.append(new_name)
  elif '2015' in col:
    new_name = col.replace('2015', 'Y14')
    new_col_names.append(new_name)
  elif '2016' in col:
    new_name = col.replace('2016', 'Y15')
    new_col_names.append(new_name)
  else:
    new_col_names.append(col)

new_col_names

# apply new col names

df_predictions.columns = new_col_names

df_predictions.head()


,index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_Y1,...,O3_med_pred_Y5,O3_med_pred_Y6,O3_med_pred_Y7,O3_med_pred_Y8,O3_med_pred_Y9,O3_med_pred_Y10,O3_med_pred_Y12,O3_med_pred_Y13,O3_med_pred_Y14,O3_med_pred_Y15
0,0,11,None,11.923392,11.545831,11.460930,40.514807,39.489763,39.457052,12.869481,...,43.838816,40.486148,36.221797,41.289135,40.862455,39.302231,36.364677,37.080065,35.994939,39.567964
1,1,13,None,10.724676,9.577370,9.549603,44.482241,40.738241,40.819741,12.162325,...,43.525804,40.236525,38.505225,42.373774,42.022552,39.985824,37.438556,38.104659,36.873269,38.886460
2,2,15,None,11.368095,10.766793,10.752574,40.595697,39.667357,39.633515,11.531937,...,43.837824,41.187681,37.317680,41.459194,41.776717,39.300123,36.652613,37.449995,35.411200,38.890084
3,3,17,None,11.830653,11.281247,11.299146,40.101601,39.013332,39.004118,13.162910,...,43.850361,39.875838,35.059151,39.827738,39.680734,38.873873,35.961584,36.127933,35.970548,39.368079
4,4,19,None,12.847922,12.097639,12.144016,42.326738,40.748248,40.746380,14.722058,...,45.038788,41.869574,37.462589,42.542686,41.728563,41.646953,37.739042,38.077132,37.006927,41.614854


In [23]:
X_predictions = df_predictions.drop(['index', 'FIPS', 'recent_trend'], axis=1)

predictions_scaler =StandardScaler().fit(X_predictions)
X_predictions_scaled =predictions_scaler.transform(X_predictions)

prediction_trend= brc.predict(X_predictions_scaled)

df_predictions['prediction_trend'] = prediction_trend

df_predictions.head()

,index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_Y1,...,O3_med_pred_Y6,O3_med_pred_Y7,O3_med_pred_Y8,O3_med_pred_Y9,O3_med_pred_Y10,O3_med_pred_Y12,O3_med_pred_Y13,O3_med_pred_Y14,O3_med_pred_Y15,prediction_trend
0,0,11,None,11.923392,11.545831,11.460930,40.514807,39.489763,39.457052,12.869481,...,40.486148,36.221797,41.289135,40.862455,39.302231,36.364677,37.080065,35.994939,39.567964,stable
1,1,13,None,10.724676,9.577370,9.549603,44.482241,40.738241,40.819741,12.162325,...,40.236525,38.505225,42.373774,42.022552,39.985824,37.438556,38.104659,36.873269,38.886460,falling
2,2,15,None,11.368095,10.766793,10.752574,40.595697,39.667357,39.633515,11.531937,...,41.187681,37.317680,41.459194,41.776717,39.300123,36.652613,37.449995,35.411200,38.890084,stable
3,3,17,None,11.830653,11.281247,11.299146,40.101601,39.013332,39.004118,13.162910,...,39.875838,35.059151,39.827738,39.680734,38.873873,35.961584,36.127933,35.970548,39.368079,stable
4,4,19,None,12.847922,12.097639,12.144016,42.326738,40.748248,40.746380,14.722058,...,41.869574,37.462589,42.542686,41.728563,41.646953,37.739042,38.077132,37.006927,41.614854,stable


In [24]:
pred_data = df_predictions[['FIPS', 'prediction_trend']].copy()

pred_data.head()

,FIPS,prediction_trend
0,11,stable
1,13,falling
2,15,stable
3,17,stable
4,19,stable


In [25]:
pred_data.to_sql('prediction_results', engine, if_exists='replace', index=False)